In [ ]:
import os
import json
import random
from pathlib import Path
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from datetime import datetime
import time
from scipy.stats import pearsonr

from sklearn.metrics import r2_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

# ---------------------------------------------------------------------------- #
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
set_seed(42)

In [ ]:
# ---------------------------------------------------------------------------- #
# 配置与参数
# ---------------------------------------------------------------------------- #
INPUT_FOLDERS = {
    'u': r'Y:/meso-unet/40km/2025-01-05/mean_u',
    'v': r'Y:/meso-unet/40km/2025-01-05/mean_v',
    't': r'Y:/meso-unet/40km/2025-01-05/mean_t',
    'q': r'Y:/meso-unet/40km/2025-01-05/mean_q',
    'wz': r'Y:/meso-unet/40km/2025-01-05/mean_wz',
}
TARGET_FOLDERS = {
    'uw': r'Y:/meso-unet/40km/2025-01-05/uw',
    'vw': r'Y:/meso-unet/40km/2025-01-05/vw',
    'tw': r'Y:/meso-unet/40km/2025-01-05/tw',
    'qw': r'Y:/meso-unet/40km/2025-01-05/qw'
}
TERRAIN_PATH = r'F:/data/terrain/zs.npy'
STD_PATH = r'F:/data/terrain/std.npy'
TARGET_STATS_PATH = "./json/target_stats_40km-1.json"
INPUT_STATS_PATH = "./json/input_stats_40km-1.json"
IN_CHANNELS = 7
OUT_CHANNELS = 4
DEPTH = 64 
HEIGHT, WIDTH = 20, 20
BASE_CHANNELS = 128
BATCH_SIZE = 128
NUM_WORKERS = 0
LR = 1e-4
NUM_EPOCHS = 200
TRAIN_VAL_SPLIT = 0.8
MAX_SAMPLES = 100000
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
CHECKPOINT_DIR = Path(r'F:/model/model_save')
CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)

PATIENCE_ES = 20
DZ = 0.1
LAMBDA_GRAD = 0.1
CHANNELS = ['uw', 'vw', 'tw', 'qw']
# ---------------------------------------------------------------------------- #
# 2) 统计函数：计算目标字段全局均值与标准差
# ---------------------------------------------------------------------------- #
def compute_target_stats_layerwise(keys, target_folders, depth=64):
    """
    每个目标变量按高度层计算均值和标准差。
    返回 dict: means[var] 和 stds[var] 为 shape=(depth,) 的列表
    """
    means = {}
    stds = {}
    for tgt, folder in target_folders.items():
        folder = Path(folder)
        sum_layer = np.zeros(depth)
        sumsq_layer = np.zeros(depth)
        count_layer = 0
        for key in tqdm(keys, desc=f"Target stats (layerwise) - {tgt}"):
            arr3d = np.load(folder / f"{key}.npy", mmap_mode='r')[:, :, :depth]
            flat = arr3d.reshape(-1, depth)
            sum_layer += flat.sum(axis=0)
            sumsq_layer += (flat ** 2).sum(axis=0)
            count_layer += flat.shape[0]
        μ = sum_layer / count_layer
        σ = np.sqrt(sumsq_layer / count_layer - μ ** 2)
        stds[tgt] = np.where(σ < 1e-3, 1e-3, σ).astype(np.float32).tolist()
        means[tgt] = μ.astype(np.float32).tolist()
    return means, stds


# ---------------------------------------------------------------------------- #
# 统计函数：计算输入字段的全局均值与标准差（按高度分层）
# ---------------------------------------------------------------------------- #
def compute_input_stats(keys, input_folders, depth=DEPTH):
    """
    计算训练集 input 各变量、各高度层的均值与 std，用于标准化。
    返回 dict: means[var]: shape(depth,), stds[var]: shape(depth,)
    """
    sums = {var: np.zeros(depth, dtype=np.float64) for var in input_folders}
    sumsqs = {var: np.zeros(depth, dtype=np.float64) for var in input_folders}
    counts = {var: 0 for var in input_folders}
    for var, folder in input_folders.items():
        folder = Path(folder)
        for key in tqdm(keys, desc=f"Input stats {var}"):
            arr3d = np.load(folder / f"{key}.npy", mmap_mode='r').astype(np.float32)  # (H,W,51)
            arr = arr3d[:, :, :depth]  # (100,100,depth)
            arr_flat = arr.reshape(-1, depth)  # (100*100, depth)
            sums[var] += arr_flat.sum(axis=0)
            sumsqs[var] += (arr_flat**2).sum(axis=0)
            counts[var] += arr_flat.shape[0]
    means = {}
    stds = {}
    for var in input_folders:
        μ = sums[var] / counts[var]  # shape (depth,)
        σ = np.sqrt(sumsqs[var] / counts[var] - μ**2)
        σ = np.where(σ < 1e-3, 1e-3, σ)
        means[var] = μ.astype(np.float32).tolist()
        stds[var] = σ.astype(np.float32).tolist()
    return means, stds

# ---------------------------------------------------------------------------- #
# 3) Dataset 定义：加载 memmap，加上全局归一化
# ---------------------------------------------------------------------------- #
class VerticalProfileDataset(Dataset):
    def __init__(self, keys, input_folders, target_folders,
                 terrain_path, std_path,
                 input_means, input_stds,
                 target_means, target_stds,
                 max_samples=100000):
        """
        input_means/stds: dict var -> list of length DEPTH
        target_means/stds: dict tgt -> float
        """
        self.keys = keys
        self.in_folders = {k: Path(v) for k, v in input_folders.items()}
        self.tg_folders = {k: Path(v) for k, v in target_folders.items()}

        # 预加载 memmap：input 和 target
        self.input_data = {var: {} for var in input_folders}
        self.target_data = {tgt: {} for tgt in target_folders}
        for var in input_folders:
            folder = self.in_folders[var]
            for key in keys:
                path = folder / f"{key}.npy"
                try:
                    arr = np.load(path, mmap_mode='r').astype(np.float32)
                except Exception as e:
                    raise RuntimeError(f"Load input npy failed: {path}") from e
                self.input_data[var][key] = arr  # shape (H,W,51)
        for tgt in target_folders:
            folder = self.tg_folders[tgt]
            for key in keys:
                path = folder / f"{key}.npy"
                try:
                    arr = np.load(path, mmap_mode='r').astype(np.float32)
                except Exception as e:
                    raise RuntimeError(f"Load target npy failed: {path}") from e
                self.target_data[tgt][key] = arr  # shape (H,W,51)

        # 载入地形与地形std，并做全局归一化
        terrain = np.load(terrain_path).astype(np.float32)  # (H,W)
        terrain_std = np.load(std_path).astype(np.float32)  # (H,W)
        t_mean = float(np.mean(terrain))
        t_std = float(np.std(terrain)); t_std = max(t_std, 1e-3)
        ts_mean = float(np.mean(terrain_std))
        ts_std = float(np.std(terrain_std)); ts_std = max(ts_std, 1e-3)
        self.terrain = (terrain - t_mean) / t_std
        self.terrain_std = (terrain_std - ts_mean) / ts_std

        # 保存 input/target 归一化参数
        self.input_means = {var: np.array(input_means[var], dtype=np.float32) for var in input_means}
        self.input_stds = {var: np.array(input_stds[var], dtype=np.float32) for var in input_stds}
        self.target_means = {var: np.array(target_means[var], dtype=np.float32) for var in target_means}
        self.target_stds = {var: np.array(target_stds[var], dtype=np.float32) for var in target_stds}

        # self.target_means = target_means
        # self.target_stds = target_stds

        # 生成所有可能的位置索引 (key, i, j)
        self.locations = []
        for key in keys:
            for i in range(HEIGHT):
                for j in range(WIDTH):
                    self.locations.append((key, i, j))
        random.shuffle(self.locations)
        self.locations = self.locations[:max_samples]
        print(f"Created dataset with {len(self.locations)} vertical profiles")

    def __len__(self):
        return len(self.locations)

    def __getitem__(self, idx):
        key, i, j = self.locations[idx]

        # 加载输入变量 (u, v, t, q, wz)，并按全局分层归一化
        input_arrays = []
        for var in ['u', 'v', 't', 'q', 'wz']:
            arr3d = self.input_data[var][key]
            arr = arr3d[i, j, :DEPTH].astype(np.float32)
            mean = self.input_means[var]
            std = self.input_stds[var]
            arr_norm = (arr - mean) / std
            input_arrays.append(arr_norm)

        # 添加地形和地形标准差
        terrain_val = float(self.terrain[i, j])
        std_val = float(self.terrain_std[i, j])
        input_arrays.append(np.full(DEPTH, terrain_val, dtype=np.float32))
        input_arrays.append(np.full(DEPTH, std_val, dtype=np.float32))

        x = torch.tensor(np.stack(input_arrays, axis=0), dtype=torch.float32)

        # 加载目标变量并归一化
        target_arrays = []
        for tgt in CHANNELS:
            arr3d = self.target_data[tgt][key]
            arr = arr3d[i, j, :DEPTH].astype(np.float32)
            μ = np.array(self.target_means[tgt], dtype=np.float32)  # shape (64,)
            σ = np.array(self.target_stds[tgt], dtype=np.float32)
            arr_norm = (arr - μ) / σ
            target_arrays.append(arr_norm)
        y = torch.tensor(np.stack(target_arrays, axis=0), dtype=torch.float32)
        return x, y

# ---------------------------------------------------------------------------- #
# 4) 1D U-Net 模块
# ---------------------------------------------------------------------------- #
class DoubleConv1D(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv1d(in_ch, out_ch, 3, padding=1, bias=False),
            nn.BatchNorm1d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv1d(out_ch, out_ch, 3, padding=1, bias=False),
            nn.BatchNorm1d(out_ch),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.double_conv(x)

class Down1D(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.seq = nn.Sequential(
            nn.MaxPool1d(2),
            DoubleConv1D(in_ch, out_ch)
        )

    def forward(self, x):
        return self.seq(x)

class Up1D(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.up = nn.ConvTranspose1d(in_ch, out_ch, kernel_size=2, stride=2)
        self.conv = DoubleConv1D(out_ch * 2, out_ch)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        diff = x2.size(2) - x1.size(2)
        if diff > 0:
            pad_left = diff // 2
            pad_right = diff - pad_left
            x1 = nn.functional.pad(x1, [pad_left, pad_right])
        elif diff < 0:
            crop = -diff
            crop_left = crop // 2
            crop_right = crop - crop_left
            x1 = x1[:, :, crop_left: x1.size(2)-crop_right]
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

class OutConv1D(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Conv1d(in_ch, out_ch, 1)

    def forward(self, x):
        return self.conv(x)

class UNet1D(nn.Module):
    def __init__(self, in_c, out_c, base=64):
        super().__init__()
        self.inc = DoubleConv1D(in_c, base)
        self.down1 = Down1D(base, base * 2)
        self.down2 = Down1D(base * 2, base * 4)
        self.down3 = Down1D(base * 4, base * 8)

        self.bottleneck = DoubleConv1D(base * 8, base * 16)

        self.up3 = Up1D(base * 16, base * 8)
        self.up2 = Up1D(base * 8, base * 4)
        self.up1 = Up1D(base * 4, base * 2)
        self.up0 = Up1D(base * 2, base)

        self.outc = OutConv1D(base, out_c)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.bottleneck(x4)
        x = self.up3(x5, x4)
        x = self.up2(x, x3)
        x = self.up1(x, x2)
        x = self.up0(x, x1)
        return self.outc(x)

# ---------------------------------------------------------------------------- #
# 5) 损失函数：加权MSE + 梯度损失
# ---------------------------------------------------------------------------- #
def compute_rmse_per_channel(pred, target):
    """
    pred, target: torch.Tensor, shape (B, C, L)
    返回 numpy array, shape (C,), per-channel RMSE averaged over batch和长度
    """
    with torch.no_grad():
        diff2 = (pred - target).cpu().numpy() ** 2
    sum_sq = diff2.sum(axis=(0, 2))
    B, C, L = diff2.shape
    rmse = np.sqrt(sum_sq / (B * L))
    return rmse

def weighted_huber_loss(pred, target, weight, delta=1.0):
    """
    pred, target: shape (B, C, L)
    weight: shape (C,)
    delta: float, Huber loss 的阈值参数
    """
    abs_diff = torch.abs(pred - target)
    quadratic = torch.minimum(abs_diff, torch.tensor(delta, device=pred.device))
    linear = abs_diff - quadratic
    loss = 0.5 * quadratic ** 2 + delta * linear
    loss = loss * weight.view(1, -1, 1)  # 每个通道加权
    return loss.mean()


# ---------------------------------------------------------------------------- #
# 6) 训练与验证函数
# ---------------------------------------------------------------------------- #
def train_one_epoch(model, loader, optimizer, device, loss_weights):
    model.train()
    running_loss = 0.0
    running_value_loss = 0.0
    for x_batch, y_batch in tqdm(loader, desc="Training", leave=False):
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(x_batch)
        value_loss = weighted_huber_loss(outputs, y_batch, loss_weights, delta=1.0)
        loss = value_loss
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * x_batch.size(0)
        running_value_loss += value_loss.item() * x_batch.size(0)
    epoch_loss = running_loss / len(loader.dataset)
    epoch_value = running_value_loss / len(loader.dataset)
    return epoch_loss, epoch_value

def plot_scatter_per_channel(preds, trues, channel_names, epoch, save_dir, max_samples=500000):
    """
    preds, trues: numpy arrays, shape [N, C, L]
    """
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, f'uvtscatter_epoch{epoch:03d}.png')
    
    C = preds.shape[1]
    fig, axes = plt.subplots(1, C, figsize=(5 * C, 5), dpi=150)
    if C == 1:
        axes = [axes]

    for c in range(C):
        ax = axes[c]
        pred_flat = preds[:, c, :].reshape(-1)
        true_flat = trues[:, c, :].reshape(-1)

        # 随机采样以提高效率
        if len(pred_flat) > max_samples:
            idx = np.random.choice(len(pred_flat), max_samples, replace=False)
            pred_flat = pred_flat[idx]
            true_flat = true_flat[idx]

        # 相关统计
        rmse = np.sqrt(np.mean((pred_flat - true_flat)**2))
        corr = pearsonr(pred_flat, true_flat)[0]
        bias = np.mean(pred_flat - true_flat)
        slope, intercept = np.polyfit(true_flat, pred_flat, 1)

        ax.hexbin(true_flat, pred_flat, gridsize=100, cmap='Spectral_r', bins='log', linewidths=0.2, mincnt=1)
        ax.plot([-1, 1], [-1, 1], 'r--', lw=1.2, alpha=0.6)
        ax.axhline(0, color='gray', linestyle='--', lw=0.5)
        ax.axvline(0, color='gray', linestyle='--', lw=0.5)

        ax.set_title(f"{channel_names[c]}\nRMSE={rmse:.3f}, Corr={corr:.3f}, Bias={bias:.3f}", fontsize=10)
        ax.set_xlabel("Truth")
        ax.set_ylabel("Pred")

    plt.suptitle(f"Scatter of Prediction vs Truth (Epoch {epoch})", fontsize=14)
    plt.tight_layout()
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()
    print(f"  Saved scatter plot to {save_path}")

def plot_r2_profile_per_layer(r2_by_layer, channel_names, epoch, save_dir):
    """
    绘制每个通道的R²随高度变化的图
    r2_by_layer: numpy array, shape [C, L]
    """
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, f"uvtr2_layer_epoch{epoch:03d}.png")

    plt.figure(figsize=(6, 6), dpi=200)
    z_levels = list(range(r2_by_layer.shape[1]))[::-1]  # 63 -> 0

    for idx, ch in enumerate(channel_names):
        plt.plot(r2_by_layer[idx][::-1], z_levels, label=ch, linewidth=2)

    plt.xlabel(r'$R^2$', fontsize=12)
    plt.ylabel('Height Level', fontsize=12)
    plt.title(f'Validation $R^2$ per Layer (Epoch {epoch})', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.08),
               ncol=len(channel_names), frameon=False, fontsize=10)
    plt.xlim(0, 1)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.tight_layout()
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()
    print(f"  Saved R² plot to {save_path}")

def validate_one_epoch(model, loader, device, loss_weights, epoch=None, writer=None, save_plot_dir=None):
    model.eval()
    running_loss = 0.0
    running_value_loss = 0.0
    sum_sq = np.zeros(OUT_CHANNELS, dtype=np.float64)
    total_count = 0

    # 收集所有预测和真实值
    all_preds = []
    all_trues = []

    with torch.no_grad():
        for x_batch, y_batch in tqdm(loader, desc="Validation", leave=False):
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            outputs = model(x_batch)

            value_loss = weighted_huber_loss(outputs, y_batch, loss_weights, delta=1.0)
            loss = value_loss

            running_loss += loss.item() * x_batch.size(0)
            running_value_loss += value_loss.item() * x_batch.size(0)

            diff2 = (outputs - y_batch).cpu().numpy() ** 2
            B, C, L = diff2.shape
            sum_sq += diff2.sum(axis=(0, 2))
            total_count += B * L

            all_preds.append(outputs.cpu())
            all_trues.append(y_batch.cpu())

    overall_loss = running_loss / len(loader.dataset)
    overall_value = running_value_loss / len(loader.dataset)
    per_channel_rmse = np.sqrt(sum_sq / total_count)

    # ===== 计算 R² 和 Pearson 散点相关系数 ===== #
    all_preds = torch.cat(all_preds, dim=0).numpy()  # shape (N, C, L)
    all_trues = torch.cat(all_trues, dim=0).numpy()

    def compute_r2_per_layer(pred, true):
        B, C, L = pred.shape
        r2_layers = np.empty((C, L))
        for c in range(C):
            for l in range(L):
                y_true = true[:, c, l]
                y_pred = pred[:, c, l]
                ss_res = np.sum((y_true - y_pred) ** 2)
                ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
                r2_layers[c, l] = 1 - ss_res / ss_tot if ss_tot > 0 else np.nan
        return r2_layers

    def compute_global_corr(pred, true):
        return np.corrcoef(pred.reshape(-1), true.reshape(-1))[0, 1]

    r2_by_layer = compute_r2_per_layer(all_preds, all_trues)  # shape (4, 64)
    global_corr = compute_global_corr(all_preds, all_trues)

    print(f"  Global Pearson Correlation: {global_corr:.4f}")
    for idx, ch in enumerate(CHANNELS):
        print(f"  R² layer mean - {ch}: {np.nanmean(r2_by_layer[idx]):.4f}")


    if save_plot_dir and epoch is not None:
        
        os.makedirs(save_plot_dir, exist_ok=True)
        plot_r2_profile_per_layer(r2_by_layer, CHANNELS, epoch, save_plot_dir)
        plot_scatter_per_channel(all_preds, all_trues, CHANNELS, epoch, save_plot_dir)

    return overall_loss, overall_value, per_channel_rmse


# ---------------------------------------------------------------------------- #\
Path("./model_plots").mkdir(parents=True, exist_ok=True)
writer = SummaryWriter(log_dir=f"runs/unet1d_exp_{datetime.now().strftime('%m%d_%H%M')}")

# 7) 主流程
# ---------------------------------------------------------------------------- #
if __name__ == "__main__":

    def denormalize(y_norm, mean, std):
        return y_norm * std + mean
    def visualize_profile_comparison(x_batch, y_true, y_pred, target_means, target_stds, idx=0):
        """
        可视化第 idx 个样本的每个通道 profile，预测 vs 观测
        """
        B, C, L = y_true.shape
        fig, axs = plt.subplots(1, C, figsize=(16, 4))
        z = np.arange(L)

        for c in range(C):
            ch = CHANNELS[c]
            mean = np.array(target_means[ch])
            std = np.array(target_stds[ch])

            true_profile = denormalize(y_true[idx, c], mean, std)
            pred_profile = denormalize(y_pred[idx, c], mean, std)

            axs[c].plot(true_profile, z, label='Truth', color='black')
            axs[c].plot(pred_profile, z, label='Pred', color='red', linestyle='--')
            axs[c].invert_yaxis()
            axs[c].set_title(ch)
            axs[c].set_xlabel('Value')
            axs[c].set_ylabel('Height')

        plt.suptitle(f"Profile Comparison for Sample {idx}")
        plt.legend()
        plt.tight_layout()
        plt.show()

    # 获取所有样本 keys
    all_keys = sorted([p.stem for p in Path(INPUT_FOLDERS['u']).glob("*.npy")])
    random.shuffle(all_keys)
    n_train = int(len(all_keys) * TRAIN_VAL_SPLIT)
    # 获取所有样本 keys
    all_keys = sorted([p.stem for p in Path(INPUT_FOLDERS['u']).glob("*.npy")])
    random.shuffle(all_keys)

    n_total = len(all_keys)
    n_train = int(n_total * 0.7)
    n_val = int(n_total * 0.15)
    n_test = n_total - n_train - n_val

    train_keys = all_keys[:n_train]
    val_keys = all_keys[n_train:n_train + n_val]
    test_keys = all_keys[n_train + n_val:]

    print(f"Total samples: {n_total}, Train: {len(train_keys)}, Val: {len(val_keys)}, Test: {len(test_keys)}")
    # 1) 计算并加载 target 统计量
    if Path(TARGET_STATS_PATH).exists():
        with open(TARGET_STATS_PATH) as f:
            stats = json.load(f)
        target_means = stats['means']
        target_stds = stats['stds']
        print("Loaded existing target stats")
    else:
        means, stds = compute_target_stats_layerwise(train_keys, TARGET_FOLDERS, depth=DEPTH)
        stats = {"means": means, "stds": stds}
        with open(TARGET_STATS_PATH, "w") as f:
            json.dump(stats, f, indent=2)
        target_means = means
        target_stds = stds
        print(f"Saved target stats to {TARGET_STATS_PATH}")

    # 2) 计算并加载 input 统计量
    if Path(INPUT_STATS_PATH).exists():
        with open(INPUT_STATS_PATH) as f:
            input_stats = json.load(f)
        input_means = input_stats['means']
        input_stds = input_stats['stds']
        print("Loaded existing input stats")
    else:
        means_in, stds_in = compute_input_stats(train_keys, INPUT_FOLDERS, depth=DEPTH)
        input_stats = {"means": means_in, "stds": stds_in}
        with open(INPUT_STATS_PATH, "w") as f:
            json.dump(input_stats, f, indent=2)
        input_means = means_in
        input_stds = stds_in
        print(f"Saved input stats to {INPUT_STATS_PATH}")

    # 3) 创建 Dataset
    train_ds = VerticalProfileDataset(
        train_keys, INPUT_FOLDERS, TARGET_FOLDERS,
        TERRAIN_PATH, STD_PATH,
        input_means, input_stds,
        target_means, target_stds,
        max_samples=MAX_SAMPLES
    )
    val_ds = VerticalProfileDataset(
        val_keys, INPUT_FOLDERS, TARGET_FOLDERS,
        TERRAIN_PATH, STD_PATH,
        input_means, input_stds,
        target_means, target_stds,
        max_samples=int(MAX_SAMPLES * 0.15)
    )
    test_ds = VerticalProfileDataset(
        test_keys, INPUT_FOLDERS, TARGET_FOLDERS,
        TERRAIN_PATH, STD_PATH,
        input_means, input_stds,
        target_means, target_stds,
        max_samples=int(MAX_SAMPLES * 0.15)
    )

    print(f"Train samples: {len(train_ds)}, Validation samples: {len(val_ds)}")

    # 4) DataLoader
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                            num_workers=NUM_WORKERS, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False,
                            num_workers=NUM_WORKERS, pin_memory=True)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False,
                            num_workers=NUM_WORKERS, pin_memory=True)

    # 5) 初始化模型、优化器、日志
    model = UNet1D(IN_CHANNELS, OUT_CHANNELS, base=BASE_CHANNELS).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)
    writer = SummaryWriter(log_dir="runs/unet1d_experiment")

    total_params = sum(p.numel() for p in model.parameters()) / 1e6
    print(f"Model initialized on {DEVICE}, total parameters: {total_params:.2f}M")

    inverse_std = 1.0 / np.array([np.mean(target_stds[ch]) for ch in CHANNELS])
    norm_weights = inverse_std / np.max(inverse_std)
    LOSS_WEIGHTS = torch.tensor(norm_weights, dtype=torch.float32).to(DEVICE)

    print(f"Auto-calculated LOSS_WEIGHTS: {LOSS_WEIGHTS}")

    # 6) 训练循环（无梯度损失）
    best_val_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(1, NUM_EPOCHS + 1):
        print(f"\nEpoch {epoch}/{NUM_EPOCHS}")

        train_loss, train_value_loss = train_one_epoch(model, train_loader, optimizer, DEVICE, LOSS_WEIGHTS)

        print(f"  Train Total Loss: {train_loss:.6f}, Value Loss: {train_value_loss:.6f}")

        # 验证阶段，包含 R² 和相关系数
        val_loss, val_value_loss, per_channel_rmse = validate_one_epoch(
            model, val_loader, DEVICE, LOSS_WEIGHTS,
            epoch=epoch,
            writer=writer,
            save_plot_dir="./model_plots"
        )
        # 打印 RMSE
        print(f"  Val Total Loss: {val_loss:.6f}, Value Loss: {val_value_loss:.6f}")
        print(f"  Per-channel RMSE: uw={per_channel_rmse[0]:.4f}, vw={per_channel_rmse[1]:.4f}, tw={per_channel_rmse[2]:.4f}, ")
        # Tensorboard 写入（Loss 和 RMSE）
        writer.add_scalar('Loss/train_total', train_loss, epoch)
        writer.add_scalar('Loss/train_value', train_value_loss, epoch)
        writer.add_scalar('Loss/val_total', val_loss, epoch)
        writer.add_scalar('Loss/val_value', val_value_loss, epoch)
        for idx, ch in enumerate(CHANNELS):
            writer.add_scalar(f'RMSE/val_{ch}', per_channel_rmse[idx], epoch)

        scheduler.step(val_loss)

        # 保存最佳模型
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
            ckpt_path = CHECKPOINT_DIR / f"best_epoch{epoch:03d}_val{val_loss:.6f}.pth"
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': val_loss,
                'input_stats': {"means": input_means, "stds": input_stds},
                'target_stats': {"means": target_means, "stds": target_stds},
            }, ckpt_path)
            print(f"  Saved best model: {ckpt_path}")
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= PATIENCE_ES:
                print(f"Early stopping at epoch {epoch} (no improvement in {PATIENCE_ES} epochs)")
                break

    print("\n===== Final Evaluation on Test Set =====")
    model.eval()
    all_preds, all_targets = [], []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Test Inference"):
            inputs, targets = batch
            inputs = inputs.to(DEVICE)
            targets = targets.to(DEVICE)

            outputs = model(inputs)
            all_preds.append(outputs)
            all_targets.append(targets)

    all_preds = torch.cat(all_preds, dim=0)  # [N, C, Z]
    all_targets = torch.cat(all_targets, dim=0)

    # 7) 保存最终模型
    final_path = CHECKPOINT_DIR / "unet1d_40km_0807.pth"
    torch.save({
        'model_state_dict': model.state_dict(),
        'input_stats': {"means": input_means, "stds": input_stds},
        'target_stats': {"means": target_means, "stds": target_stds},
    }, final_path)
    print(f"\nTraining complete. Final model saved to:\n  {final_path}")

    writer.close()



Total samples: 104, Train: 72, Val: 15, Test: 17
Loaded existing target stats
Loaded existing input stats
Created dataset with 28800 vertical profiles
Created dataset with 6000 vertical profiles
Created dataset with 6800 vertical profiles
Train samples: 28800, Validation samples: 6000
Model initialized on cuda, total parameters: 43.25M
Auto-calculated LOSS_WEIGHTS: tensor([0.0266, 0.0460, 0.0214, 1.0000], device='cuda:0')

Epoch 1/200


  Train Total Loss: 0.006563, Value Loss: 0.006563


  Global Pearson Correlation: 0.3588
  R² layer mean - uw: 0.1608
  R² layer mean - vw: 0.1310
  R² layer mean - tw: 0.1261
  R² layer mean - qw: -2110.6589
  Saved R² plot to ./model_plots\uvtr2_layer_epoch001.png
  Saved scatter plot to ./model_plots\uvtscatter_epoch001.png
  Val Total Loss: 0.005529, Value Loss: 0.005529
  Per-channel RMSE: uw=0.9143, vw=0.9278, tw=0.9282, 
  Saved best model: F:\model\model_save\best_epoch001_val0.005529.pth

Epoch 2/200


  Train Total Loss: 0.005386, Value Loss: 0.005386


  Global Pearson Correlation: 0.4668
  R² layer mean - uw: 0.2488
  R² layer mean - vw: 0.1497
  R² layer mean - tw: 0.2336
  R² layer mean - qw: -2076.0449
  Saved R² plot to ./model_plots\uvtr2_layer_epoch002.png
  Saved scatter plot to ./model_plots\uvtscatter_epoch002.png
  Val Total Loss: 0.005360, Value Loss: 0.005360
  Per-channel RMSE: uw=0.8671, vw=0.9130, tw=0.8656, 
  Saved best model: F:\model\model_save\best_epoch002_val0.005360.pth

Epoch 3/200


...

  Global Pearson Correlation: 0.9572
  R² layer mean - uw: 0.9212
  R² layer mean - vw: 0.8984
  R² layer mean - tw: 0.9265
  R² layer mean - qw: -37.4790
  Saved R² plot to ./model_plots\uvtr2_layer_epoch199.png
  Saved scatter plot to ./model_plots\uvtscatter_epoch199.png
  Val Total Loss: 0.000902, Value Loss: 0.000902
  Per-channel RMSE: uw=0.2791, vw=0.3152, tw=0.2671, 

Epoch 200/200
  Saved scatter plot to ./model_plots\uvtscatter_epoch199.png
  Val Total Loss: 0.000902, Value Loss: 0.000902
  Per-channel RMSE: uw=0.2791, vw=0.3152, tw=0.2671, 

Epoch 200/200


  Train Total Loss: 0.000169, Value Loss: 0.000169


  Global Pearson Correlation: 0.9575
  R² layer mean - uw: 0.9216
  R² layer mean - vw: 0.9000
  R² layer mean - tw: 0.9271
  R² layer mean - qw: -37.7145
  Saved R² plot to ./model_plots\uvtr2_layer_epoch200.png
  Saved scatter plot to ./model_plots\uvtscatter_epoch200.png
  Val Total Loss: 0.000893, Value Loss: 0.000893
  Per-channel RMSE: uw=0.2784, vw=0.3126, tw=0.2660, 

===== Final Evaluation on Test Set =====
  Saved scatter plot to ./model_plots\uvtscatter_epoch200.png
  Val Total Loss: 0.000893, Value Loss: 0.000893
  Per-channel RMSE: uw=0.2784, vw=0.3126, tw=0.2660, 

===== Final Evaluation on Test Set =====


Test Inference: 100%|██████████| 54/54 [00:05<00:00, 10.42it/s]


Training complete. Final model saved to:
  F:\model\model_save\unet1d_40km_0807.pth


In [ ]:
# 相比于上面的代码，增加了训练数据量，把q和qw仅仅取了[:20],输入内容显示在了log里面，散点图显示范围缩小
# ---------------------------------------------------------------------------- #
# 配置与参数
# ---------------------------------------------------------------------------- #
INPUT_FOLDERS = {
    'u': r'C:/gather/mean_u',
    'v': r'C:/gather/mean_v',
    't': r'C:/gather/mean_t',
    'q': r'C:/gather/mean_q',
    'wz': r'C:/gather/mean_wz',
}
TARGET_FOLDERS = {
    'uw': r'C:/gather/uw',
    'vw': r'C:/gather/vw',
    'tw': r'C:/gather/tw',
    'qw': r'C:/gather/qw'
}
TERRAIN_PATH = r'F:/data/terrain/zs.npy'
STD_PATH = r'F:/data/terrain/std.npy'
TARGET_STATS_PATH = "./json/target_stats_40km-2.json"
INPUT_STATS_PATH = "./json/input_stats_40km-2.json"
IN_CHANNELS = 7
OUT_CHANNELS = 4
DEPTH = 64 
HEIGHT, WIDTH = 20, 20
BASE_CHANNELS = 128
BATCH_SIZE = 128
NUM_WORKERS = 0
LR = 1e-4
NUM_EPOCHS = 200
TRAIN_VAL_SPLIT = 0.8
MAX_SAMPLES = 100000
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
CHECKPOINT_DIR = Path(r'F:/model/model_save')
CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)

PATIENCE_ES = 20
DZ = 0.1
LAMBDA_GRAD = 0.1
CHANNELS = ['uw', 'vw', 'tw', 'qw']
# ---------------------------------------------------------------------------- #
# 2) 统计函数：计算目标字段全局均值与标准差
# ---------------------------------------------------------------------------- #
def compute_target_stats_layerwise(keys, target_folders, depth=64):
    """
    每个目标变量按高度层计算均值和标准差。
    返回 dict: means[var] 和 stds[var] 为 shape=(depth,) 的列表
    """
    means = {}
    stds = {}
    for tgt, folder in target_folders.items():
        folder = Path(folder)
        sum_layer = np.zeros(depth)
        sumsq_layer = np.zeros(depth)
        count_layer = 0
        for key in tqdm(keys, desc=f"Target stats (layerwise) - {tgt}"):
            arr3d = np.load(folder / f"{key}.npy", mmap_mode='r')[:, :, :depth]
            flat = arr3d.reshape(-1, depth)
            sum_layer += flat.sum(axis=0)
            sumsq_layer += (flat ** 2).sum(axis=0)
            count_layer += flat.shape[0]
        μ = sum_layer / count_layer
        σ = np.sqrt(sumsq_layer / count_layer - μ ** 2)
        stds[tgt] = np.where(σ < 1e-3, 1e-3, σ).astype(np.float32).tolist()
        means[tgt] = μ.astype(np.float32).tolist()
    return means, stds


# ---------------------------------------------------------------------------- #
# 统计函数：计算输入字段的全局均值与标准差（按高度分层）
# ---------------------------------------------------------------------------- #
def compute_input_stats(keys, input_folders, depth=DEPTH):
    """
    计算训练集 input 各变量、各高度层的均值与 std，用于标准化。
    返回 dict: means[var]: shape(depth,), stds[var]: shape(depth,)
    """
    sums = {var: np.zeros(depth, dtype=np.float64) for var in input_folders}
    sumsqs = {var: np.zeros(depth, dtype=np.float64) for var in input_folders}
    counts = {var: 0 for var in input_folders}
    for var, folder in input_folders.items():
        folder = Path(folder)
        for key in tqdm(keys, desc=f"Input stats {var}"):
            arr3d = np.load(folder / f"{key}.npy", mmap_mode='r').astype(np.float32)  # (H,W,51)
            arr = arr3d[:, :, :depth]  # (100,100,depth)
            arr_flat = arr.reshape(-1, depth)  # (100*100, depth)
            sums[var] += arr_flat.sum(axis=0)
            sumsqs[var] += (arr_flat**2).sum(axis=0)
            counts[var] += arr_flat.shape[0]
    means = {}
    stds = {}
    for var in input_folders:
        μ = sums[var] / counts[var]  # shape (depth,)
        σ = np.sqrt(sumsqs[var] / counts[var] - μ**2)
        σ = np.where(σ < 1e-3, 1e-3, σ)
        means[var] = μ.astype(np.float32).tolist()
        stds[var] = σ.astype(np.float32).tolist()
    return means, stds

# ---------------------------------------------------------------------------- #
# 3) Dataset 定义：加载 memmap，加上全局归一化
# ---------------------------------------------------------------------------- #
class VerticalProfileDataset(Dataset):
    def __init__(self, keys, input_folders, target_folders,
                 terrain_path, std_path,
                 input_means, input_stds,
                 target_means, target_stds,
                 max_samples=100000):
        """
        input_means/stds: dict var -> list of length DEPTH
        target_means/stds: dict tgt -> float
        """
        self.keys = keys
        self.in_folders = {k: Path(v) for k, v in input_folders.items()}
        self.tg_folders = {k: Path(v) for k, v in target_folders.items()}

        # 预加载 memmap：input 和 target
        self.input_data = {var: {} for var in input_folders}
        self.target_data = {tgt: {} for tgt in target_folders}
        for var in input_folders:
            folder = self.in_folders[var]
            for key in keys:
                path = folder / f"{key}.npy"
                try:
                    arr = np.load(path, mmap_mode='r').astype(np.float32)
                except Exception as e:
                    raise RuntimeError(f"Load input npy failed: {path}") from e
                self.input_data[var][key] = arr  # shape (H,W,51)
        for tgt in target_folders:
            folder = self.tg_folders[tgt]
            for key in keys:
                path = folder / f"{key}.npy"
                try:
                    arr = np.load(path, mmap_mode='r').astype(np.float32)
                except Exception as e:
                    raise RuntimeError(f"Load target npy failed: {path}") from e
                self.target_data[tgt][key] = arr  # shape (H,W,51)

        # 载入地形与地形std，并做全局归一化
        terrain = np.load(terrain_path).astype(np.float32)  # (H,W)
        terrain_std = np.load(std_path).astype(np.float32)  # (H,W)
        t_mean = float(np.mean(terrain))
        t_std = float(np.std(terrain)); t_std = max(t_std, 1e-3)
        ts_mean = float(np.mean(terrain_std))
        ts_std = float(np.std(terrain_std)); ts_std = max(ts_std, 1e-3)
        self.terrain = (terrain - t_mean) / t_std
        self.terrain_std = (terrain_std - ts_mean) / ts_std

        # 保存 input/target 归一化参数
        self.input_means = {var: np.array(input_means[var], dtype=np.float32) for var in input_means}
        self.input_stds = {var: np.array(input_stds[var], dtype=np.float32) for var in input_stds}
        self.target_means = {var: np.array(target_means[var], dtype=np.float32) for var in target_means}
        self.target_stds = {var: np.array(target_stds[var], dtype=np.float32) for var in target_stds}

        # self.target_means = target_means
        # self.target_stds = target_stds

        # 生成所有可能的位置索引 (key, i, j)
        self.locations = []
        for key in keys:
            for i in range(HEIGHT):
                for j in range(WIDTH):
                    self.locations.append((key, i, j))
        random.shuffle(self.locations)
        self.locations = self.locations[:max_samples]
        print(f"Created dataset with {len(self.locations)} vertical profiles")
        log_print(f"Created dataset with {len(self.locations)} vertical profiles")
    def __len__(self):
        return len(self.locations)

    def __getitem__(self, idx):
        key, i, j = self.locations[idx]

        # 加载输入变量 (u, v, t, q, wz)，并按全局分层归一化
        input_arrays = []
        for var in ['u', 'v', 't', 'q', 'wz']:
            arr3d = self.input_data[var][key]
            arr = arr3d[i, j, :DEPTH].astype(np.float32)
            mean = self.input_means[var]
            std = self.input_stds[var]
            arr_norm = (arr - mean) / std
            if var == 'q':
                arr_norm[20:] = 0.0
            input_arrays.append(arr_norm)

        # 添加地形和地形标准差
        terrain_val = float(self.terrain[i, j])
        std_val = float(self.terrain_std[i, j])
        input_arrays.append(np.full(DEPTH, terrain_val, dtype=np.float32))
        input_arrays.append(np.full(DEPTH, std_val, dtype=np.float32))

        x = torch.tensor(np.stack(input_arrays, axis=0), dtype=torch.float32)

        # 加载目标变量并归一化
        target_arrays = []
        for tgt in CHANNELS:
            arr3d = self.target_data[tgt][key]
            arr = arr3d[i, j, :DEPTH].astype(np.float32)
            μ = np.array(self.target_means[tgt], dtype=np.float32)  # shape (64,)
            σ = np.array(self.target_stds[tgt], dtype=np.float32)
            arr_norm = (arr - μ) / σ
            if tgt == 'qw':
                arr_norm[20:] = 0.0
            target_arrays.append(arr_norm)
        y = torch.tensor(np.stack(target_arrays, axis=0), dtype=torch.float32)
        return x, y

# ---------------------------------------------------------------------------- #
# 4) 1D U-Net 模块
# ---------------------------------------------------------------------------- #
class DoubleConv1D(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv1d(in_ch, out_ch, 3, padding=1, bias=False),
            nn.BatchNorm1d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv1d(out_ch, out_ch, 3, padding=1, bias=False),
            nn.BatchNorm1d(out_ch),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.double_conv(x)

class Down1D(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.seq = nn.Sequential(
            nn.MaxPool1d(2),
            DoubleConv1D(in_ch, out_ch)
        )

    def forward(self, x):
        return self.seq(x)

class Up1D(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.up = nn.ConvTranspose1d(in_ch, out_ch, kernel_size=2, stride=2)
        self.conv = DoubleConv1D(out_ch * 2, out_ch)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        diff = x2.size(2) - x1.size(2)
        if diff > 0:
            pad_left = diff // 2
            pad_right = diff - pad_left
            x1 = nn.functional.pad(x1, [pad_left, pad_right])
        elif diff < 0:
            crop = -diff
            crop_left = crop // 2
            crop_right = crop - crop_left
            x1 = x1[:, :, crop_left: x1.size(2)-crop_right]
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

class OutConv1D(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Conv1d(in_ch, out_ch, 1)

    def forward(self, x):
        return self.conv(x)

class UNet1D(nn.Module):
    def __init__(self, in_c, out_c, base=64):
        super().__init__()
        self.inc = DoubleConv1D(in_c, base)
        self.down1 = Down1D(base, base * 2)
        self.down2 = Down1D(base * 2, base * 4)
        self.down3 = Down1D(base * 4, base * 8)

        self.bottleneck = DoubleConv1D(base * 8, base * 16)

        self.up3 = Up1D(base * 16, base * 8)
        self.up2 = Up1D(base * 8, base * 4)
        self.up1 = Up1D(base * 4, base * 2)
        self.up0 = Up1D(base * 2, base)

        self.outc = OutConv1D(base, out_c)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.bottleneck(x4)
        x = self.up3(x5, x4)
        x = self.up2(x, x3)
        x = self.up1(x, x2)
        x = self.up0(x, x1)
        return self.outc(x)

# ---------------------------------------------------------------------------- #
# 5) 损失函数：加权MSE + 梯度损失
# ---------------------------------------------------------------------------- #
def compute_rmse_per_channel(pred, target):
    """
    pred, target: torch.Tensor, shape (B, C, L)
    返回 numpy array, shape (C,), per-channel RMSE averaged over batch和长度
    """
    with torch.no_grad():
        diff2 = (pred - target).cpu().numpy() ** 2
    sum_sq = diff2.sum(axis=(0, 2))
    B, C, L = diff2.shape
    rmse = np.sqrt(sum_sq / (B * L))
    return rmse

def weighted_huber_loss(pred, target, weight, delta=1.0):
    """
    pred, target: shape (B, C, L)
    weight: shape (C,)
    delta: float, Huber loss 的阈值参数
    """
    abs_diff = torch.abs(pred - target)
    quadratic = torch.minimum(abs_diff, torch.tensor(delta, device=pred.device))
    linear = abs_diff - quadratic
    loss = 0.5 * quadratic ** 2 + delta * linear
    loss = loss * weight.view(1, -1, 1)
    # 对qw通道，只计算前20层损失
    qw_idx = CHANNELS.index('qw')
    loss[:, qw_idx, 20:] = 0.0
    return loss.mean()

# ---------------------------------------------------------------------------- #
# 6) 训练与验证函数
# ---------------------------------------------------------------------------- #
def train_one_epoch(model, loader, optimizer, device, loss_weights):
    model.train()
    running_loss = 0.0
    running_value_loss = 0.0
    for x_batch, y_batch in tqdm(loader, desc="Training", leave=False):
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(x_batch)
        value_loss = weighted_huber_loss(outputs, y_batch, loss_weights, delta=1.0)
        loss = value_loss
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * x_batch.size(0)
        running_value_loss += value_loss.item() * x_batch.size(0)
    epoch_loss = running_loss / len(loader.dataset)
    epoch_value = running_value_loss / len(loader.dataset)
    return epoch_loss, epoch_value

def plot_scatter_per_channel(preds, trues, channel_names, epoch, save_dir, max_samples=500000):
    """
    绘制预测与真实的散点图，去除极端值，坐标轴对称，绘制1:1线和拟合线
    """
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, f'uvtscatter_epoch{epoch:03d}.png')
    C = preds.shape[1]
    fig, axes = plt.subplots(1, C, figsize=(5 * C, 5), dpi=150)
    if C == 1:
        axes = [axes]

    for c in range(C):
        ax = axes[c]
        if channel_names[c] == 'qw':
            pred_flat = preds[:, c, :20].reshape(-1)
            true_flat = trues[:, c, :20].reshape(-1)
        else:
            pred_flat = preds[:, c, :].reshape(-1)
            true_flat = trues[:, c, :].reshape(-1)

        # 随机采样以提高效率
        if len(pred_flat) > max_samples:
            idx = np.random.choice(len(pred_flat), max_samples, replace=False)
            pred_flat = pred_flat[idx]
            true_flat = true_flat[idx]

        # 去除极端值（最大/最小千分之一）
        q_low = 0.001
        q_high = 0.999
        x_min, x_max = np.quantile(true_flat, [q_low, q_high])
        y_min, y_max = np.quantile(pred_flat, [q_low, q_high])
        lim = max(abs(x_min), abs(x_max), abs(y_min), abs(y_max))
        lim = np.ceil(lim * 10) / 10  # 向上取整到0.1

        # 只保留在范围内的数据
        mask = (true_flat >= -lim) & (true_flat <= lim) & (pred_flat >= -lim) & (pred_flat <= lim)
        true_flat = true_flat[mask]
        pred_flat = pred_flat[mask]

        # 相关统计
        rmse = np.sqrt(np.mean((pred_flat - true_flat)**2))
        corr = pearsonr(pred_flat, true_flat)[0]
        bias = np.mean(pred_flat - true_flat)
        slope, intercept = np.polyfit(true_flat, pred_flat, 1)

        # 绘制散点
        ax.hexbin(true_flat, pred_flat, gridsize=100, cmap='Spectral_r', bins='log', linewidths=0.2, mincnt=1)
        # 1:1参考线
        ax.plot([-lim, lim], [-lim, lim], 'r--', lw=1.2, alpha=0.6, label='1:1')
        # 拟合线
        x_fit = np.array([-lim, lim])
        y_fit = slope * x_fit + intercept
        ax.plot(x_fit, y_fit, 'b-', lw=1.2, alpha=0.7, label=f'Fit: y={slope:.2f}x+{intercept:.2f}')

        ax.axhline(0, color='gray', linestyle='--', lw=0.5)
        ax.axvline(0, color='gray', linestyle='--', lw=0.5)

        ax.set_xlim(-lim, lim)
        ax.set_ylim(-lim, lim)
        ax.set_aspect('equal', adjustable='box')
        ax.set_title(f"{channel_names[c]}\nRMSE={rmse:.3f}, Corr={corr:.3f}, Bias={bias:.3f}", fontsize=10)
        ax.set_xlabel("Truth")
        ax.set_ylabel("Pred")
        ax.legend(fontsize=8, loc='upper left')

    plt.suptitle(f"Scatter of Prediction vs Truth (Epoch {epoch})", fontsize=14)
    plt.tight_layout()
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()
    print(f"  Saved scatter plot to {save_path}")
def plot_r2_profile_per_layer(r2_by_layer, channel_names, epoch, save_dir):
    """
    绘制每个通道的R²随高度变化的图
    r2_by_layer: numpy array, shape [C, L]
    """
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, f"uvtr2_layer_epoch{epoch:03d}.png")

    plt.figure(figsize=(6, 6), dpi=200)
    z_levels = list(range(r2_by_layer.shape[1]))[::-1]  # 63 -> 0

    for idx, ch in enumerate(channel_names):
        if ch == 'qw':
            plt.plot(r2_by_layer[idx][:20][::-1], z_levels[-20:], label=ch, linewidth=2)
        else:
            plt.plot(r2_by_layer[idx][::-1], z_levels, label=ch, linewidth=2)

    plt.xlabel(r'$R^2$', fontsize=12)
    plt.ylabel('Height Level', fontsize=12)
    plt.title(f'Validation $R^2$ per Layer (Epoch {epoch})', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.08),
               ncol=len(channel_names), frameon=False, fontsize=10)
    plt.xlim(0, 1)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.tight_layout()
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()
    print(f"  Saved R² plot to {save_path}")

def validate_one_epoch(model, loader, device, loss_weights, epoch=None, writer=None, save_plot_dir=None):
    model.eval()
    running_loss = 0.0
    running_value_loss = 0.0
    sum_sq = np.zeros(OUT_CHANNELS, dtype=np.float64)
    total_count = np.zeros(OUT_CHANNELS, dtype=np.float64)

    all_preds = []
    all_trues = []

    with torch.no_grad():
        for x_batch, y_batch in tqdm(loader, desc="Validation", leave=False):
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            outputs = model(x_batch)

            value_loss = weighted_huber_loss(outputs, y_batch, loss_weights, delta=1.0)
            loss = value_loss

            running_loss += loss.item() * x_batch.size(0)
            running_value_loss += value_loss.item() * x_batch.size(0)

            diff2 = (outputs - y_batch).cpu().numpy() ** 2
            B, C, L = diff2.shape
            for c, ch in enumerate(CHANNELS):
                if ch == 'qw':
                    sum_sq[c] += diff2[:, c, :20].sum()
                    total_count[c] += B * 20
                else:
                    sum_sq[c] += diff2[:, c, :].sum()
                    total_count[c] += B * L

            all_preds.append(outputs.cpu())
            all_trues.append(y_batch.cpu())

    overall_loss = running_loss / len(loader.dataset)
    overall_value = running_value_loss / len(loader.dataset)
    per_channel_rmse = np.sqrt(sum_sq / total_count)

    # ===== 计算 R² 和 Pearson 散点相关系数 ===== #
    all_preds = torch.cat(all_preds, dim=0).numpy()  # shape (N, C, L)
    all_trues = torch.cat(all_trues, dim=0).numpy()

    def compute_r2_per_layer(pred, true):
        B, C, L = pred.shape
        r2_layers = np.empty((C, L))
        for c in range(C):
            for l in range(L):
                if CHANNELS[c] == 'qw' and l >= 20:
                    r2_layers[c, l] = np.nan
                    continue
                y_true = true[:, c, l]
                y_pred = pred[:, c, l]
                ss_res = np.sum((y_true - y_pred) ** 2)
                ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
                r2_layers[c, l] = 1 - ss_res / ss_tot if ss_tot > 0 else np.nan
        return r2_layers

    def compute_global_corr(pred, true):
        return np.corrcoef(pred.reshape(-1), true.reshape(-1))[0, 1]

    r2_by_layer = compute_r2_per_layer(all_preds, all_trues)  # shape (4, 64)
    global_corr = compute_global_corr(all_preds, all_trues)

    print(f"  Global Pearson Correlation: {global_corr:.4f}")
    for idx, ch in enumerate(CHANNELS):
        print(f"  R² layer mean - {ch}: {np.nanmean(r2_by_layer[idx]):.4f}")
        log_print(f"  R² layer mean - {ch}: {np.nanmean(r2_by_layer[idx]):.4f}")

    if save_plot_dir and epoch is not None:
        
        os.makedirs(save_plot_dir, exist_ok=True)
        plot_r2_profile_per_layer(r2_by_layer, CHANNELS, epoch, save_plot_dir)
        plot_scatter_per_channel(all_preds, all_trues, CHANNELS, epoch, save_plot_dir)

    return overall_loss, overall_value, per_channel_rmse


# ---------------------------------------------------------------------------- #\
Path("./model_plots1").mkdir(parents=True, exist_ok=True)
writer = SummaryWriter(log_dir=f"runs/unet1d_exp_{datetime.now().strftime('%m%d_%H%M')}")

# 7) 主流程
# ---------------------------------------------------------------------------- #
if __name__ == "__main__":
    LOG_PATH = "./train_log.txt"
    log_file = open(LOG_PATH, "w", encoding="utf-8")

    def log_print(*args, **kwargs):
        print(*args, **kwargs, file=log_file)
        log_file.flush()
        
    def denormalize(y_norm, mean, std):
        return y_norm * std + mean

    # 获取所有样本 keys
    all_keys = sorted([p.stem for p in Path(INPUT_FOLDERS['u']).glob("*.npy")])
    random.shuffle(all_keys)
    n_train = int(len(all_keys) * TRAIN_VAL_SPLIT)
    # 获取所有样本 keys
    all_keys = sorted([p.stem for p in Path(INPUT_FOLDERS['u']).glob("*.npy")])
    random.shuffle(all_keys)

    n_total = len(all_keys)
    n_train = int(n_total * 0.7)
    n_val = int(n_total * 0.15)
    n_test = n_total - n_train - n_val

    train_keys = all_keys[:n_train]
    val_keys = all_keys[n_train:n_train + n_val]
    test_keys = all_keys[n_train + n_val:]

    print(f"Total samples: {n_total}, Train: {len(train_keys)}, Val: {len(val_keys)}, Test: {len(test_keys)}")
    log_print(f"Total samples: {n_total}, Train: {len(train_keys)}, Val: {len(val_keys)}, Test: {len(test_keys)}")

    # 1) 计算并加载 target 统计量
    if Path(TARGET_STATS_PATH).exists():
        with open(TARGET_STATS_PATH) as f:
            stats = json.load(f)
        target_means = stats['means']
        target_stds = stats['stds']
        print("Loaded existing target stats")
    else:
        means, stds = compute_target_stats_layerwise(train_keys, TARGET_FOLDERS, depth=DEPTH)
        stats = {"means": means, "stds": stds}
        with open(TARGET_STATS_PATH, "w") as f:
            json.dump(stats, f, indent=2)
        target_means = means
        target_stds = stds
        print(f"Saved target stats to {TARGET_STATS_PATH}")

    # 2) 计算并加载 input 统计量
    if Path(INPUT_STATS_PATH).exists():
        with open(INPUT_STATS_PATH) as f:
            input_stats = json.load(f)
        input_means = input_stats['means']
        input_stds = input_stats['stds']
        print("Loaded existing input stats")
    else:
        means_in, stds_in = compute_input_stats(train_keys, INPUT_FOLDERS, depth=DEPTH)
        input_stats = {"means": means_in, "stds": stds_in}
        with open(INPUT_STATS_PATH, "w") as f:
            json.dump(input_stats, f, indent=2)
        input_means = means_in
        input_stds = stds_in
        print(f"Saved input stats to {INPUT_STATS_PATH}")

    # 3) 创建 Dataset
    train_ds = VerticalProfileDataset(
        train_keys, INPUT_FOLDERS, TARGET_FOLDERS,
        TERRAIN_PATH, STD_PATH,
        input_means, input_stds,
        target_means, target_stds,
        max_samples=MAX_SAMPLES
    )
    val_ds = VerticalProfileDataset(
        val_keys, INPUT_FOLDERS, TARGET_FOLDERS,
        TERRAIN_PATH, STD_PATH,
        input_means, input_stds,
        target_means, target_stds,
        max_samples=int(MAX_SAMPLES * 0.15)
    )
    test_ds = VerticalProfileDataset(
        test_keys, INPUT_FOLDERS, TARGET_FOLDERS,
        TERRAIN_PATH, STD_PATH,
        input_means, input_stds,
        target_means, target_stds,
        max_samples=int(MAX_SAMPLES * 0.15)
    )

    print(f"Train samples: {len(train_ds)}, Validation samples: {len(val_ds)}")
    log_print(f"Train samples: {len(train_ds)}, Validation samples: {len(val_ds)}")    
    # 4) DataLoader
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                            num_workers=NUM_WORKERS, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False,
                            num_workers=NUM_WORKERS, pin_memory=True)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False,
                            num_workers=NUM_WORKERS, pin_memory=True)

    # 5) 初始化模型、优化器、日志
    model = UNet1D(IN_CHANNELS, OUT_CHANNELS, base=BASE_CHANNELS).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)
    writer = SummaryWriter(log_dir="runs/unet1d_experiment")

    total_params = sum(p.numel() for p in model.parameters()) / 1e6
    print(f"Model initialized on {DEVICE}, total parameters: {total_params:.2f}M")
    log_print(f"Model initialized on {DEVICE}, total parameters: {total_params:.2f}M")
    inverse_std = 1.0 / np.array([np.mean(target_stds[ch]) for ch in CHANNELS])
    norm_weights = inverse_std / np.max(inverse_std)
    LOSS_WEIGHTS = torch.tensor(norm_weights, dtype=torch.float32).to(DEVICE)

    print(f"Auto-calculated LOSS_WEIGHTS: {LOSS_WEIGHTS}")
    log_print(f"Auto-calculated LOSS_WEIGHTS: {LOSS_WEIGHTS}")

    # 6) 训练循环（无梯度损失）
    best_val_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(1, NUM_EPOCHS + 1):
        print(f"\nEpoch {epoch}/{NUM_EPOCHS}")
        log_print(f"\nEpoch {epoch}/{NUM_EPOCHS}")
        train_loss, train_value_loss = train_one_epoch(model, train_loader, optimizer, DEVICE, LOSS_WEIGHTS)

        print(f"  Train Total Loss: {train_loss:.6f}, Value Loss: {train_value_loss:.6f}")
        log_print(f"  Train Total Loss: {train_loss:.6f}, Value Loss: {train_value_loss:.6f}")
        # 验证阶段，包含 R² 和相关系数
        val_loss, val_value_loss, per_channel_rmse = validate_one_epoch(
            model, val_loader, DEVICE, LOSS_WEIGHTS,
            epoch=epoch,
            writer=writer,
            save_plot_dir="./model_plots1"
        )
        # 打印 RMSE
        print(f"  Val Total Loss: {val_loss:.6f}, Value Loss: {val_value_loss:.6f}")
        print(f"  Per-channel RMSE: uw={per_channel_rmse[0]:.4f}, vw={per_channel_rmse[1]:.4f}, tw={per_channel_rmse[2]:.4f}, ")
        log_print(f"  Val Total Loss: {val_loss:.6f}, Value Loss: {val_value_loss:.6f}")
        log_print(f"  Per-channel RMSE: uw={per_channel_rmse[0]:.4f}, vw={per_channel_rmse[1]:.4f}, tw={per_channel_rmse[2]:.4f}, ")

        # Tensorboard 写入（Loss 和 RMSE）
        writer.add_scalar('Loss/train_total', train_loss, epoch)
        writer.add_scalar('Loss/train_value', train_value_loss, epoch)
        writer.add_scalar('Loss/val_total', val_loss, epoch)
        writer.add_scalar('Loss/val_value', val_value_loss, epoch)
        for idx, ch in enumerate(CHANNELS):
            writer.add_scalar(f'RMSE/val_{ch}', per_channel_rmse[idx], epoch)

        scheduler.step(val_loss)

        # 保存最佳模型
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
            ckpt_path = CHECKPOINT_DIR / f"best_epoch{epoch:03d}_val{val_loss:.6f}.pth"
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': val_loss,
                'input_stats': {"means": input_means, "stds": input_stds},
                'target_stats': {"means": target_means, "stds": target_stds},
            }, ckpt_path)
            print(f"  Saved best model: {ckpt_path}")
            log_print(f"  Saved best model: {ckpt_path}")
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= PATIENCE_ES:
                print(f"Early stopping at epoch {epoch} (no improvement in {PATIENCE_ES} epochs)")
                log_print(f"Early stopping at epoch {epoch} (no improvement in {PATIENCE_ES} epochs)")

                break

    print("\n===== Final Evaluation on Test Set =====")
    log_print("\n===== Final Evaluation on Test Set =====")
    model.eval()
    all_preds, all_targets = [], []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Test Inference"):
            inputs, targets = batch
            inputs = inputs.to(DEVICE)
            targets = targets.to(DEVICE)

            outputs = model(inputs)
            all_preds.append(outputs)
            all_targets.append(targets)

    all_preds = torch.cat(all_preds, dim=0)  # [N, C, Z]
    all_targets = torch.cat(all_targets, dim=0)

    # 7) 保存最终模型
    final_path = CHECKPOINT_DIR / "unet1d_40km_0807-gather.pth"
    torch.save({
        'model_state_dict': model.state_dict(),
        'input_stats': {"means": input_means, "stds": input_stds},
        'target_stats': {"means": target_means, "stds": target_stds},
    }, final_path)
    print(f"\nTraining complete. Final model saved to:\n  {final_path}")
    log_print(f"\nTraining complete. Final model saved to:\n  {final_path}")
    writer.close()



Total samples: 469, Train: 328, Val: 70, Test: 71
Loaded existing target stats
Loaded existing input stats
Created dataset with 100000 vertical profiles
Created dataset with 15000 vertical profiles
Created dataset with 15000 vertical profiles
Train samples: 100000, Validation samples: 15000
Model initialized on cuda, total parameters: 43.25M
Auto-calculated LOSS_WEIGHTS: tensor([0.0158, 0.0404, 0.0183, 1.0000], device='cuda:0')

Epoch 1/200


  Train Total Loss: 0.003819, Value Loss: 0.003819


  Global Pearson Correlation: 0.5662
  R² layer mean - uw: 0.3433
  R² layer mean - vw: 0.2599
  R² layer mean - tw: 0.3192
  R² layer mean - qw: -5.7947
  Saved R² plot to ./model_plots1\uvtr2_layer_epoch001.png
  Saved scatter plot to ./model_plots1\uvtscatter_epoch001.png
  Val Total Loss: 0.003745, Value Loss: 0.003745
  Per-channel RMSE: uw=0.8834, vw=0.9457, tw=0.8855, 
  Saved best model: F:\model\model_save\best_epoch001_val0.003745.pth

Epoch 2/200


  Train Total Loss: 0.003212, Value Loss: 0.003212


  Global Pearson Correlation: 0.6080
  R² layer mean - uw: 0.3646
  R² layer mean - vw: 0.3019
  R² layer mean - tw: 0.2988
  R² layer mean - qw: -5.1133
  Saved R² plot to ./model_plots1\uvtr2_layer_epoch002.png
  Saved scatter plot to ./model_plots1\uvtscatter_epoch002.png
  Val Total Loss: 0.003862, Value Loss: 0.003862
  Per-channel RMSE: uw=0.8712, vw=0.9183, tw=0.8985, 

Epoch 3/200


...

  Global Pearson Correlation: 0.9476
  R² layer mean - uw: 0.9066
  R² layer mean - vw: 0.8850
  R² layer mean - tw: 0.8986
  R² layer mean - qw: 0.6857
  Saved R² plot to ./model_plots1\uvtr2_layer_epoch199.png
  Saved scatter plot to ./model_plots1\uvtscatter_epoch199.png
  Val Total Loss: 0.000961, Value Loss: 0.000961
  Per-channel RMSE: uw=0.3320, vw=0.3705, tw=0.3418, 

Epoch 200/200


  Train Total Loss: 0.000263, Value Loss: 0.000263


  Global Pearson Correlation: 0.9475
  R² layer mean - uw: 0.9065
  R² layer mean - vw: 0.8855
  R² layer mean - tw: 0.8991
  R² layer mean - qw: 0.6842
  Saved R² plot to ./model_plots1\uvtr2_layer_epoch200.png
  Saved scatter plot to ./model_plots1\uvtscatter_epoch200.png
  Val Total Loss: 0.000960, Value Loss: 0.000960
  Per-channel RMSE: uw=0.3321, vw=0.3696, tw=0.3411, 

===== Final Evaluation on Test Set =====


Test Inference: 100%|██████████| 118/118 [00:11<00:00, 10.30it/s]


Training complete. Final model saved to:
  F:\model\model_save\unet1d_40km_0807-gather.pth


In [ ]:
# 相比于上面的代码，增加了训练数据量，把q和qw仅仅取了[:20],输入内容显示在了log里面，散点图显示范围缩小
# ---------------------------------------------------------------------------- #
# 配置与参数
# ---------------------------------------------------------------------------- #
INPUT_FOLDERS = {
    'u': r'C:/gather/mean_u',
    'v': r'C:/gather/mean_v',
    't': r'C:/gather/mean_t',
    'q': r'C:/gather/mean_q',
    'wz': r'C:/gather/mean_wz',
}
TARGET_FOLDERS = {
    'uw': r'C:/gather/uw',
    'vw': r'C:/gather/vw',
    'tw': r'C:/gather/tw',
    'qw': r'C:/gather/qw'
}
TERRAIN_PATH = r'F:/data/terrain/zs.npy'
STD_PATH = r'F:/data/terrain/std.npy'
TARGET_STATS_PATH = "./json/target_stats_40km-2.json"
INPUT_STATS_PATH = "./json/input_stats_40km-2.json"
IN_CHANNELS = 7
OUT_CHANNELS = 4
DEPTH = 64 
HEIGHT, WIDTH = 20, 20
BASE_CHANNELS = 128
BATCH_SIZE = 128
NUM_WORKERS = 0
LR = 1e-4
NUM_EPOCHS = 200
TRAIN_VAL_SPLIT = 0.8
MAX_SAMPLES = 100000
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
CHECKPOINT_DIR = Path(r'F:/model/model_save')
CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)

PATIENCE_ES = 20
DZ = 0.1
LAMBDA_GRAD = 0.1
CHANNELS = ['uw', 'vw', 'tw', 'qw']
# ---------------------------------------------------------------------------- #
# 2) 统计函数：计算目标字段全局均值与标准差
# ---------------------------------------------------------------------------- #
def compute_target_stats_layerwise(keys, target_folders, depth=64):
    """
    每个目标变量按高度层计算均值和标准差。
    返回 dict: means[var] 和 stds[var] 为 shape=(depth,) 的列表
    """
    means = {}
    stds = {}
    for tgt, folder in target_folders.items():
        folder = Path(folder)
        sum_layer = np.zeros(depth)
        sumsq_layer = np.zeros(depth)
        count_layer = 0
        for key in tqdm(keys, desc=f"Target stats (layerwise) - {tgt}"):
            arr3d = np.load(folder / f"{key}.npy", mmap_mode='r')[:, :, :depth]
            flat = arr3d.reshape(-1, depth)
            sum_layer += flat.sum(axis=0)
            sumsq_layer += (flat ** 2).sum(axis=0)
            count_layer += flat.shape[0]
        μ = sum_layer / count_layer
        σ = np.sqrt(sumsq_layer / count_layer - μ ** 2)
        stds[tgt] = np.where(σ < 1e-3, 1e-3, σ).astype(np.float32).tolist()
        means[tgt] = μ.astype(np.float32).tolist()
    return means, stds


# ---------------------------------------------------------------------------- #
# 统计函数：计算输入字段的全局均值与标准差（按高度分层）
# ---------------------------------------------------------------------------- #
def compute_input_stats(keys, input_folders, depth=DEPTH):
    """
    计算训练集 input 各变量、各高度层的均值与 std，用于标准化。
    返回 dict: means[var]: shape(depth,), stds[var]: shape(depth,)
    """
    sums = {var: np.zeros(depth, dtype=np.float64) for var in input_folders}
    sumsqs = {var: np.zeros(depth, dtype=np.float64) for var in input_folders}
    counts = {var: 0 for var in input_folders}
    for var, folder in input_folders.items():
        folder = Path(folder)
        for key in tqdm(keys, desc=f"Input stats {var}"):
            arr3d = np.load(folder / f"{key}.npy", mmap_mode='r').astype(np.float32)  # (H,W,51)
            arr = arr3d[:, :, :depth]  # (100,100,depth)
            arr_flat = arr.reshape(-1, depth)  # (100*100, depth)
            sums[var] += arr_flat.sum(axis=0)
            sumsqs[var] += (arr_flat**2).sum(axis=0)
            counts[var] += arr_flat.shape[0]
    means = {}
    stds = {}
    for var in input_folders:
        μ = sums[var] / counts[var]  # shape (depth,)
        σ = np.sqrt(sumsqs[var] / counts[var] - μ**2)
        σ = np.where(σ < 1e-3, 1e-3, σ)
        means[var] = μ.astype(np.float32).tolist()
        stds[var] = σ.astype(np.float32).tolist()
    return means, stds

# ---------------------------------------------------------------------------- #
# 3) Dataset 定义：加载 memmap，加上全局归一化
# ---------------------------------------------------------------------------- #
class VerticalProfileDataset(Dataset):
    def __init__(self, keys, input_folders, target_folders,
                 terrain_path, std_path,
                 input_means, input_stds,
                 target_means, target_stds,
                 max_samples=100000):
        """
        input_means/stds: dict var -> list of length DEPTH
        target_means/stds: dict tgt -> float
        """
        self.keys = keys
        self.in_folders = {k: Path(v) for k, v in input_folders.items()}
        self.tg_folders = {k: Path(v) for k, v in target_folders.items()}

        # 预加载 memmap：input 和 target
        self.input_data = {var: {} for var in input_folders}
        self.target_data = {tgt: {} for tgt in target_folders}
        for var in input_folders:
            folder = self.in_folders[var]
            for key in keys:
                path = folder / f"{key}.npy"
                try:
                    arr = np.load(path, mmap_mode='r').astype(np.float32)
                except Exception as e:
                    raise RuntimeError(f"Load input npy failed: {path}") from e
                self.input_data[var][key] = arr  # shape (H,W,51)
        for tgt in target_folders:
            folder = self.tg_folders[tgt]
            for key in keys:
                path = folder / f"{key}.npy"
                try:
                    arr = np.load(path, mmap_mode='r').astype(np.float32)
                except Exception as e:
                    raise RuntimeError(f"Load target npy failed: {path}") from e
                self.target_data[tgt][key] = arr  # shape (H,W,51)

        # 载入地形与地形std，并做全局归一化
        terrain = np.load(terrain_path).astype(np.float32)  # (H,W)
        terrain_std = np.load(std_path).astype(np.float32)  # (H,W)
        t_mean = float(np.mean(terrain))
        t_std = float(np.std(terrain)); t_std = max(t_std, 1e-3)
        ts_mean = float(np.mean(terrain_std))
        ts_std = float(np.std(terrain_std)); ts_std = max(ts_std, 1e-3)
        self.terrain = (terrain - t_mean) / t_std
        self.terrain_std = (terrain_std - ts_mean) / ts_std

        # 保存 input/target 归一化参数
        self.input_means = {var: np.array(input_means[var], dtype=np.float32) for var in input_means}
        self.input_stds = {var: np.array(input_stds[var], dtype=np.float32) for var in input_stds}
        self.target_means = {var: np.array(target_means[var], dtype=np.float32) for var in target_means}
        self.target_stds = {var: np.array(target_stds[var], dtype=np.float32) for var in target_stds}

        # self.target_means = target_means
        # self.target_stds = target_stds

        # 生成所有可能的位置索引 (key, i, j)
        self.locations = []
        for key in keys:
            for i in range(HEIGHT):
                for j in range(WIDTH):
                    self.locations.append((key, i, j))
        random.shuffle(self.locations)
        self.locations = self.locations[:max_samples]
        print(f"Created dataset with {len(self.locations)} vertical profiles")
        log_print(f"Created dataset with {len(self.locations)} vertical profiles")
    def __len__(self):
        return len(self.locations)

    def __getitem__(self, idx):
        key, i, j = self.locations[idx]

        # 加载输入变量 (u, v, t, q, wz)，并按全局分层归一化
        input_arrays = []
        for var in ['u', 'v', 't', 'q', 'wz']:
            arr3d = self.input_data[var][key]
            arr = arr3d[i, j, :DEPTH].astype(np.float32)
            mean = self.input_means[var]
            std = self.input_stds[var]
            arr_norm = (arr - mean) / std
            # if var == 'q':
            #     arr_norm[20:] = 0.0
            input_arrays.append(arr_norm)

        # 添加地形和地形标准差
        terrain_val = float(self.terrain[i, j])
        std_val = float(self.terrain_std[i, j])
        input_arrays.append(np.full(DEPTH, terrain_val, dtype=np.float32))
        input_arrays.append(np.full(DEPTH, std_val, dtype=np.float32))

        x = torch.tensor(np.stack(input_arrays, axis=0), dtype=torch.float32)

        # 加载目标变量并归一化
        target_arrays = []
        for tgt in CHANNELS:
            arr3d = self.target_data[tgt][key]
            arr = arr3d[i, j, :DEPTH].astype(np.float32)
            μ = np.array(self.target_means[tgt], dtype=np.float32)  # shape (64,)
            σ = np.array(self.target_stds[tgt], dtype=np.float32)
            arr_norm = (arr - μ) / σ
            if tgt == 'qw':
                arr_norm[20:] = 0.0
            target_arrays.append(arr_norm)
        y = torch.tensor(np.stack(target_arrays, axis=0), dtype=torch.float32)
        return x, y

# ---------------------------------------------------------------------------- #
# 4) 1D U-Net 模块
# ---------------------------------------------------------------------------- #
class DoubleConv1D(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv1d(in_ch, out_ch, 3, padding=1, bias=False),
            nn.BatchNorm1d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv1d(out_ch, out_ch, 3, padding=1, bias=False),
            nn.BatchNorm1d(out_ch),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.double_conv(x)

class Down1D(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.seq = nn.Sequential(
            nn.MaxPool1d(2),
            DoubleConv1D(in_ch, out_ch)
        )

    def forward(self, x):
        return self.seq(x)

class Up1D(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.up = nn.ConvTranspose1d(in_ch, out_ch, kernel_size=2, stride=2)
        self.conv = DoubleConv1D(out_ch * 2, out_ch)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        diff = x2.size(2) - x1.size(2)
        if diff > 0:
            pad_left = diff // 2
            pad_right = diff - pad_left
            x1 = nn.functional.pad(x1, [pad_left, pad_right])
        elif diff < 0:
            crop = -diff
            crop_left = crop // 2
            crop_right = crop - crop_left
            x1 = x1[:, :, crop_left: x1.size(2)-crop_right]
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

class OutConv1D(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Conv1d(in_ch, out_ch, 1)

    def forward(self, x):
        return self.conv(x)

class UNet1D(nn.Module):
    def __init__(self, in_c, out_c, base=64):
        super().__init__()
        self.inc = DoubleConv1D(in_c, base)
        self.down1 = Down1D(base, base * 2)
        self.down2 = Down1D(base * 2, base * 4)
        self.down3 = Down1D(base * 4, base * 8)

        self.bottleneck = DoubleConv1D(base * 8, base * 16)

        self.up3 = Up1D(base * 16, base * 8)
        self.up2 = Up1D(base * 8, base * 4)
        self.up1 = Up1D(base * 4, base * 2)
        self.up0 = Up1D(base * 2, base)

        self.outc = OutConv1D(base, out_c)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.bottleneck(x4)
        x = self.up3(x5, x4)
        x = self.up2(x, x3)
        x = self.up1(x, x2)
        x = self.up0(x, x1)
        return self.outc(x)

# ---------------------------------------------------------------------------- #
# 5) 损失函数：加权MSE + 梯度损失
# ---------------------------------------------------------------------------- #
def compute_rmse_per_channel(pred, target):
    """
    pred, target: torch.Tensor, shape (B, C, L)
    返回 numpy array, shape (C,), per-channel RMSE averaged over batch和长度
    """
    with torch.no_grad():
        diff2 = (pred - target).cpu().numpy() ** 2
    sum_sq = diff2.sum(axis=(0, 2))
    B, C, L = diff2.shape
    rmse = np.sqrt(sum_sq / (B * L))
    return rmse

def weighted_huber_loss(pred, target, weight, delta=1.0):
    """
    pred, target: shape (B, C, L)
    weight: shape (C,)
    delta: float, Huber loss 的阈值参数
    """
    abs_diff = torch.abs(pred - target)
    quadratic = torch.minimum(abs_diff, torch.tensor(delta, device=pred.device))
    linear = abs_diff - quadratic
    loss = 0.5 * quadratic ** 2 + delta * linear
    loss = loss * weight.view(1, -1, 1)
    # 对qw通道，只计算前20层损失
    qw_idx = CHANNELS.index('qw')
    loss[:, qw_idx, 20:] = 0.0
    return loss.mean()

# ---------------------------------------------------------------------------- #
# 6) 训练与验证函数
# ---------------------------------------------------------------------------- #
def train_one_epoch(model, loader, optimizer, device, loss_weights):
    model.train()
    running_loss = 0.0
    running_value_loss = 0.0
    for x_batch, y_batch in tqdm(loader, desc="Training", leave=False):
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(x_batch)
        value_loss = weighted_huber_loss(outputs, y_batch, loss_weights, delta=1.0)
        loss = value_loss
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * x_batch.size(0)
        running_value_loss += value_loss.item() * x_batch.size(0)
    epoch_loss = running_loss / len(loader.dataset)
    epoch_value = running_value_loss / len(loader.dataset)
    return epoch_loss, epoch_value

def plot_scatter_per_channel(preds, trues, channel_names, epoch, save_dir, max_samples=500000):
    """
    绘制预测与真实的散点图，去除极端值，坐标轴对称，绘制1:1线和拟合线
    """
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, f'uvtscatter_epoch{epoch:03d}.png')
    C = preds.shape[1]
    fig, axes = plt.subplots(1, C, figsize=(5 * C, 5), dpi=150)
    if C == 1:
        axes = [axes]

    for c in range(C):
        ax = axes[c]
        if channel_names[c] == 'qw':
            pred_flat = preds[:, c, :20].reshape(-1)
            true_flat = trues[:, c, :20].reshape(-1)
        else:
            pred_flat = preds[:, c, :].reshape(-1)
            true_flat = trues[:, c, :].reshape(-1)

        # 随机采样以提高效率
        if len(pred_flat) > max_samples:
            idx = np.random.choice(len(pred_flat), max_samples, replace=False)
            pred_flat = pred_flat[idx]
            true_flat = true_flat[idx]

        # 去除极端值（最大/最小千分之一）
        q_low = 0.001
        q_high = 0.999
        x_min, x_max = np.quantile(true_flat, [q_low, q_high])
        y_min, y_max = np.quantile(pred_flat, [q_low, q_high])
        lim = max(abs(x_min), abs(x_max), abs(y_min), abs(y_max))
        lim = np.ceil(lim * 10) / 10  # 向上取整到0.1

        # 只保留在范围内的数据
        mask = (true_flat >= -lim) & (true_flat <= lim) & (pred_flat >= -lim) & (pred_flat <= lim)
        true_flat = true_flat[mask]
        pred_flat = pred_flat[mask]

        # 相关统计
        rmse = np.sqrt(np.mean((pred_flat - true_flat)**2))
        corr = pearsonr(pred_flat, true_flat)[0]
        bias = np.mean(pred_flat - true_flat)
        slope, intercept = np.polyfit(true_flat, pred_flat, 1)

        # 绘制散点
        ax.hexbin(true_flat, pred_flat, gridsize=100, cmap='Spectral_r', bins='log', linewidths=0.2, mincnt=1)
        # 1:1参考线
        ax.plot([-lim, lim], [-lim, lim], 'r--', lw=1.2, alpha=0.6, label='1:1')
        # 拟合线
        x_fit = np.array([-lim, lim])
        y_fit = slope * x_fit + intercept
        ax.plot(x_fit, y_fit, 'b-', lw=1.2, alpha=0.7, label=f'Fit: y={slope:.2f}x+{intercept:.2f}')

        ax.axhline(0, color='gray', linestyle='--', lw=0.5)
        ax.axvline(0, color='gray', linestyle='--', lw=0.5)

        ax.set_xlim(-lim, lim)
        ax.set_ylim(-lim, lim)
        ax.set_aspect('equal', adjustable='box')
        ax.set_title(f"{channel_names[c]}\nRMSE={rmse:.3f}, Corr={corr:.3f}, Bias={bias:.3f}", fontsize=10)
        ax.set_xlabel("Truth")
        ax.set_ylabel("Pred")
        ax.legend(fontsize=8, loc='upper left')

    plt.suptitle(f"Scatter of Prediction vs Truth (Epoch {epoch})", fontsize=14)
    plt.tight_layout()
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()
    print(f"  Saved scatter plot to {save_path}")
def plot_r2_profile_per_layer(r2_by_layer, channel_names, epoch, save_dir):
    """
    绘制每个通道的R²随高度变化的图
    r2_by_layer: numpy array, shape [C, L]
    """
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, f"uvtr2_layer_epoch{epoch:03d}.png")

    plt.figure(figsize=(6, 6), dpi=200)
    z_levels = list(range(r2_by_layer.shape[1]))[::-1]  # 63 -> 0

    for idx, ch in enumerate(channel_names):
        if ch == 'qw':
            plt.plot(r2_by_layer[idx][:20][::-1], z_levels[-20:], label=ch, linewidth=2)
        else:
            plt.plot(r2_by_layer[idx][::-1], z_levels, label=ch, linewidth=2)

    plt.xlabel(r'$R^2$', fontsize=12)
    plt.ylabel('Height Level', fontsize=12)
    plt.title(f'Validation $R^2$ per Layer (Epoch {epoch})', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.08),
               ncol=len(channel_names), frameon=False, fontsize=10)
    plt.xlim(0, 1)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.tight_layout()
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()
    print(f"  Saved R² plot to {save_path}")

def validate_one_epoch(model, loader, device, loss_weights, epoch=None, writer=None, save_plot_dir=None):
    model.eval()
    running_loss = 0.0
    running_value_loss = 0.0
    sum_sq = np.zeros(OUT_CHANNELS, dtype=np.float64)
    total_count = np.zeros(OUT_CHANNELS, dtype=np.float64)

    all_preds = []
    all_trues = []

    with torch.no_grad():
        for x_batch, y_batch in tqdm(loader, desc="Validation", leave=False):
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            outputs = model(x_batch)

            value_loss = weighted_huber_loss(outputs, y_batch, loss_weights, delta=1.0)
            loss = value_loss

            running_loss += loss.item() * x_batch.size(0)
            running_value_loss += value_loss.item() * x_batch.size(0)

            diff2 = (outputs - y_batch).cpu().numpy() ** 2
            B, C, L = diff2.shape
            for c, ch in enumerate(CHANNELS):
                if ch == 'qw':
                    sum_sq[c] += diff2[:, c, :20].sum()
                    total_count[c] += B * 20
                else:
                    sum_sq[c] += diff2[:, c, :].sum()
                    total_count[c] += B * L

            all_preds.append(outputs.cpu())
            all_trues.append(y_batch.cpu())

    overall_loss = running_loss / len(loader.dataset)
    overall_value = running_value_loss / len(loader.dataset)
    per_channel_rmse = np.sqrt(sum_sq / total_count)

    # ===== 计算 R² 和 Pearson 散点相关系数 ===== #
    all_preds = torch.cat(all_preds, dim=0).numpy()  # shape (N, C, L)
    all_trues = torch.cat(all_trues, dim=0).numpy()

    def compute_r2_per_layer(pred, true):
        B, C, L = pred.shape
        r2_layers = np.empty((C, L))
        for c in range(C):
            for l in range(L):
                if CHANNELS[c] == 'qw' and l >= 20:
                    r2_layers[c, l] = np.nan
                    continue
                y_true = true[:, c, l]
                y_pred = pred[:, c, l]
                ss_res = np.sum((y_true - y_pred) ** 2)
                ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
                r2_layers[c, l] = 1 - ss_res / ss_tot if ss_tot > 0 else np.nan
        return r2_layers

    def compute_global_corr(pred, true):
        return np.corrcoef(pred.reshape(-1), true.reshape(-1))[0, 1]

    r2_by_layer = compute_r2_per_layer(all_preds, all_trues)  # shape (4, 64)
    global_corr = compute_global_corr(all_preds, all_trues)

    print(f"  Global Pearson Correlation: {global_corr:.4f}")
    for idx, ch in enumerate(CHANNELS):
        print(f"  R² layer mean - {ch}: {np.nanmean(r2_by_layer[idx]):.4f}")
        log_print(f"  R² layer mean - {ch}: {np.nanmean(r2_by_layer[idx]):.4f}")

    if save_plot_dir and epoch is not None:
        
        os.makedirs(save_plot_dir, exist_ok=True)
        plot_r2_profile_per_layer(r2_by_layer, CHANNELS, epoch, save_plot_dir)
        plot_scatter_per_channel(all_preds, all_trues, CHANNELS, epoch, save_plot_dir)

    return overall_loss, overall_value, per_channel_rmse


# ---------------------------------------------------------------------------- #\
Path("./model_plots1").mkdir(parents=True, exist_ok=True)
writer = SummaryWriter(log_dir=f"runs/unet1d_exp_{datetime.now().strftime('%m%d_%H%M')}")

# 7) 主流程
# ---------------------------------------------------------------------------- #
if __name__ == "__main__":
    LOG_PATH = "./train_log.txt"
    log_file = open(LOG_PATH, "w", encoding="utf-8")

    def log_print(*args, **kwargs):
        print(*args, **kwargs, file=log_file)
        log_file.flush()
        
    def denormalize(y_norm, mean, std):
        return y_norm * std + mean

    # 获取所有样本 keys
    all_keys = sorted([p.stem for p in Path(INPUT_FOLDERS['u']).glob("*.npy")])
    random.shuffle(all_keys)
    n_train = int(len(all_keys) * TRAIN_VAL_SPLIT)
    # 获取所有样本 keys
    all_keys = sorted([p.stem for p in Path(INPUT_FOLDERS['u']).glob("*.npy")])
    random.shuffle(all_keys)

    n_total = len(all_keys)
    n_train = int(n_total * 0.7)
    n_val = int(n_total * 0.15)
    n_test = n_total - n_train - n_val

    train_keys = all_keys[:n_train]
    val_keys = all_keys[n_train:n_train + n_val]
    test_keys = all_keys[n_train + n_val:]

    print(f"Total samples: {n_total}, Train: {len(train_keys)}, Val: {len(val_keys)}, Test: {len(test_keys)}")
    log_print(f"Total samples: {n_total}, Train: {len(train_keys)}, Val: {len(val_keys)}, Test: {len(test_keys)}")

    # 1) 计算并加载 target 统计量
    if Path(TARGET_STATS_PATH).exists():
        with open(TARGET_STATS_PATH) as f:
            stats = json.load(f)
        target_means = stats['means']
        target_stds = stats['stds']
        print("Loaded existing target stats")
    else:
        means, stds = compute_target_stats_layerwise(train_keys, TARGET_FOLDERS, depth=DEPTH)
        stats = {"means": means, "stds": stds}
        with open(TARGET_STATS_PATH, "w") as f:
            json.dump(stats, f, indent=2)
        target_means = means
        target_stds = stds
        print(f"Saved target stats to {TARGET_STATS_PATH}")

    # 2) 计算并加载 input 统计量
    if Path(INPUT_STATS_PATH).exists():
        with open(INPUT_STATS_PATH) as f:
            input_stats = json.load(f)
        input_means = input_stats['means']
        input_stds = input_stats['stds']
        print("Loaded existing input stats")
    else:
        means_in, stds_in = compute_input_stats(train_keys, INPUT_FOLDERS, depth=DEPTH)
        input_stats = {"means": means_in, "stds": stds_in}
        with open(INPUT_STATS_PATH, "w") as f:
            json.dump(input_stats, f, indent=2)
        input_means = means_in
        input_stds = stds_in
        print(f"Saved input stats to {INPUT_STATS_PATH}")

    # 3) 创建 Dataset
    train_ds = VerticalProfileDataset(
        train_keys, INPUT_FOLDERS, TARGET_FOLDERS,
        TERRAIN_PATH, STD_PATH,
        input_means, input_stds,
        target_means, target_stds,
        max_samples=MAX_SAMPLES
    )
    val_ds = VerticalProfileDataset(
        val_keys, INPUT_FOLDERS, TARGET_FOLDERS,
        TERRAIN_PATH, STD_PATH,
        input_means, input_stds,
        target_means, target_stds,
        max_samples=int(MAX_SAMPLES * 0.15)
    )
    test_ds = VerticalProfileDataset(
        test_keys, INPUT_FOLDERS, TARGET_FOLDERS,
        TERRAIN_PATH, STD_PATH,
        input_means, input_stds,
        target_means, target_stds,
        max_samples=int(MAX_SAMPLES * 0.15)
    )

    print(f"Train samples: {len(train_ds)}, Validation samples: {len(val_ds)}")
    log_print(f"Train samples: {len(train_ds)}, Validation samples: {len(val_ds)}")    
    # 4) DataLoader
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                            num_workers=NUM_WORKERS, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False,
                            num_workers=NUM_WORKERS, pin_memory=True)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False,
                            num_workers=NUM_WORKERS, pin_memory=True)

    # 5) 初始化模型、优化器、日志
    model = UNet1D(IN_CHANNELS, OUT_CHANNELS, base=BASE_CHANNELS).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)
    writer = SummaryWriter(log_dir="runs/unet1d_experiment")

    total_params = sum(p.numel() for p in model.parameters()) / 1e6
    print(f"Model initialized on {DEVICE}, total parameters: {total_params:.2f}M")
    log_print(f"Model initialized on {DEVICE}, total parameters: {total_params:.2f}M")
    inverse_std = 1.0 / np.array([np.mean(target_stds[ch]) for ch in CHANNELS])
    norm_weights = inverse_std / np.max(inverse_std)
    LOSS_WEIGHTS = torch.tensor(norm_weights, dtype=torch.float32).to(DEVICE)

    print(f"Auto-calculated LOSS_WEIGHTS: {LOSS_WEIGHTS}")
    log_print(f"Auto-calculated LOSS_WEIGHTS: {LOSS_WEIGHTS}")

    # 6) 训练循环（无梯度损失）
    best_val_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(1, NUM_EPOCHS + 1):
        print(f"\nEpoch {epoch}/{NUM_EPOCHS}")
        log_print(f"\nEpoch {epoch}/{NUM_EPOCHS}")
        train_loss, train_value_loss = train_one_epoch(model, train_loader, optimizer, DEVICE, LOSS_WEIGHTS)

        print(f"  Train Total Loss: {train_loss:.6f}, Value Loss: {train_value_loss:.6f}")
        log_print(f"  Train Total Loss: {train_loss:.6f}, Value Loss: {train_value_loss:.6f}")
        # 验证阶段，包含 R² 和相关系数
        val_loss, val_value_loss, per_channel_rmse = validate_one_epoch(
            model, val_loader, DEVICE, LOSS_WEIGHTS,
            epoch=epoch,
            writer=writer,
            save_plot_dir="./model_plots1"
        )
        # 打印 RMSE
        print(f"  Val Total Loss: {val_loss:.6f}, Value Loss: {val_value_loss:.6f}")
        print(f"  Per-channel RMSE: uw={per_channel_rmse[0]:.4f}, vw={per_channel_rmse[1]:.4f}, tw={per_channel_rmse[2]:.4f}, ")
        log_print(f"  Val Total Loss: {val_loss:.6f}, Value Loss: {val_value_loss:.6f}")
        log_print(f"  Per-channel RMSE: uw={per_channel_rmse[0]:.4f}, vw={per_channel_rmse[1]:.4f}, tw={per_channel_rmse[2]:.4f}, ")

        # Tensorboard 写入（Loss 和 RMSE）
        writer.add_scalar('Loss/train_total', train_loss, epoch)
        writer.add_scalar('Loss/train_value', train_value_loss, epoch)
        writer.add_scalar('Loss/val_total', val_loss, epoch)
        writer.add_scalar('Loss/val_value', val_value_loss, epoch)
        for idx, ch in enumerate(CHANNELS):
            writer.add_scalar(f'RMSE/val_{ch}', per_channel_rmse[idx], epoch)

        scheduler.step(val_loss)

        # 保存最佳模型
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
            ckpt_path = CHECKPOINT_DIR / f"best_epoch{epoch:03d}_val{val_loss:.6f}.pth"
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': val_loss,
                'input_stats': {"means": input_means, "stds": input_stds},
                'target_stats': {"means": target_means, "stds": target_stds},
            }, ckpt_path)
            print(f"  Saved best model: {ckpt_path}")
            log_print(f"  Saved best model: {ckpt_path}")
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= PATIENCE_ES:
                print(f"Early stopping at epoch {epoch} (no improvement in {PATIENCE_ES} epochs)")
                log_print(f"Early stopping at epoch {epoch} (no improvement in {PATIENCE_ES} epochs)")

                break

    print("\n===== Final Evaluation on Test Set =====")
    log_print("\n===== Final Evaluation on Test Set =====")
    model.eval()
    all_preds, all_targets = [], []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Test Inference"):
            inputs, targets = batch
            inputs = inputs.to(DEVICE)
            targets = targets.to(DEVICE)

            outputs = model(inputs)
            all_preds.append(outputs)
            all_targets.append(targets)

    all_preds = torch.cat(all_preds, dim=0)  # [N, C, Z]
    all_targets = torch.cat(all_targets, dim=0)

    # 7) 保存最终模型
    final_path = CHECKPOINT_DIR / "unet1d_40km_0807-gather.pth"
    torch.save({
        'model_state_dict': model.state_dict(),
        'input_stats': {"means": input_means, "stds": input_stds},
        'target_stats': {"means": target_means, "stds": target_stds},
    }, final_path)
    print(f"\nTraining complete. Final model saved to:\n  {final_path}")
    log_print(f"\nTraining complete. Final model saved to:\n  {final_path}")
    writer.close()

